# Quality control notebook for Pasteur 2025 course

In [1]:
#@markdown ##Play the cell to connect your Google Drive to Colab

#@markdown * Click on the URL.

#@markdown * Sign in your Google Account.

#@markdown * Copy the authorization code.

#@markdown * Enter the authorization code.

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive".

# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
#@markdown ##Install dependencies

!pip install -q numpy==1.24.3 numba --no-cache-dir --force-reinstall
!pip install -q stardist


print('Stopping runtime...')
exit()
print('You can run the next cell now.')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 193.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 40.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cuda 25.2.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
albucore 0.0.24 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 which is incompatible.
distributed-ucxx-cu12 0.42.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
blosc2 3.3.2 re

In [1]:
#@markdown ##Load dependencies

import os
import numpy as np
import pandas as pd
from skimage.io import imread
from stardist.matching import matching
from tqdm import tqdm
from tabulate import tabulate
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm
from matplotlib.backends.backend_pdf import PdfPages


In [8]:
#@markdown ##Conduct QC on test data

# Define paths
gt_labels = '/content/gdrive/MyDrive/Pasteur_Image_analysis_course_2025_BiaPy/2_test/2-label'  #@param {type:"string"}
predicted_labels = '/content/gdrive/MyDrive/Pasteur_Image_analysis_course_2025_BiaPy/3_model/Pasteur_100/results/Pasteur_100_1/per_image_instances'  #@param {type:"string"}
results_folder = '/content/gdrive/MyDrive/Pasteur_Image_analysis_course_2025_BiaPy/3_model/Pasteur_100/results/Pasteur_100_1' #@param {type:"string"}

# Create the QC folder inside the results folder
QC_folder = os.path.join(results_folder, "QC")
os.makedirs(QC_folder, exist_ok=True) # Create the folder if it doesn't exist

def load_label_images_with_names(folder_path):
    file_names = sorted(f for f in os.listdir(folder_path) if f.lower().endswith(('.tif', '.tiff', '.png', '.jpg', '.jpeg')))
    return [(fn, imread(os.path.join(folder_path, fn))) for fn in file_names]

# Load data
gt_data = load_label_images_with_names(gt_labels)
pred_data = load_label_images_with_names(predicted_labels)

assert len(gt_data) == len(pred_data), "Mismatch in number of GT and predicted images."

# ---- Initialize PDF for saving visualizations ---- #
pdf_path = os.path.join(QC_folder, "QC_visualizations.pdf")
pdf = PdfPages(pdf_path)

# ---- Evaluate and visualize ---- #
records = []
print("\nEvaluating and saving visualizations...\n")

for (gt_name, gt_img), (pred_name, pred_img) in tqdm(zip(gt_data, pred_data), total=len(gt_data), desc="Processing"):
    assert gt_name == pred_name, f"Mismatched files: {gt_name} vs {pred_name}"
    res = matching(gt_img, pred_img, thresh=0.5, criterion='iou')._asdict()
    res["filename"] = gt_name

    # Calculate measured IoU for binary masks
    gt_mask = (gt_img > 0).astype(np.uint8) * 255
    pred_mask = (pred_img > 0).astype(np.uint8) * 255
    intersection = np.logical_and(gt_mask, pred_mask)
    union = np.logical_or(gt_mask, pred_mask)
    iou_score = np.sum(intersection) / np.sum(union) if np.sum(union) > 0 else 0

    res["measured_IoU"] = iou_score  # Add measured IoU to the record
    records.append(res)

    # Visualization
    plt.figure(figsize=(15, 4))
    plt.suptitle(f'{gt_name} — IoU: {iou_score:.3f}', fontsize=14)

    plt.subplot(1, 3, 1)
    plt.imshow(gt_mask, cmap='Greens')
    plt.axis('off')
    plt.title('Ground Truth')

    plt.subplot(1, 3, 2)
    plt.imshow(pred_mask, cmap='Purples')
    plt.axis('off')
    plt.title('Prediction')

    plt.subplot(1, 3, 3)
    plt.imshow(gt_mask, cmap='Greens')
    plt.imshow(pred_mask, cmap='Purples', alpha=0.5)
    plt.axis('off')
    plt.title('Overlay')

    plt.tight_layout()

    # Save figure to PDF
    pdf.savefig()
    plt.close()

# Save PDF
pdf.close()

# ---- Save results as CSV ---- #
df = pd.DataFrame(records)
cols = ['filename', 'measured_IoU'] + [col for col in df.columns if col not in ['filename', 'measured_IoU']]
df = df[cols]  # Reorder columns
csv_path = os.path.join(QC_folder, "QC_metrics.csv")
df.to_csv(csv_path, index=False)

# ---- Print final message ---- #
print("\n✅ Visualizations saved to:", pdf_path)
print("✅ Metrics CSV saved to:", csv_path)
print("\nFinal Quality Control Summary:\n")
print(tabulate(df, headers='keys', tablefmt='psql'))


Evaluating and saving visualizations...



Processing: 100%|██████████| 10/10 [00:12<00:00,  1.29s/it]



✅ Visualizations saved to: /content/gdrive/MyDrive/Pasteur_Image_analysis_course_2025_BiaPy/3_model/Pasteur_100/results/Pasteur_100_1/QC/QC_visualizations.pdf
✅ Metrics CSV saved to: /content/gdrive/MyDrive/Pasteur_Image_analysis_course_2025_BiaPy/3_model/Pasteur_100/results/Pasteur_100_1/QC/QC_metrics.csv

Final Quality Control Summary:

+----+--------------------------------------------------------------------------+----------------+-------------+----------+------+------+------+-------------+----------+------------+----------+----------+----------+-------------------+----------------------+--------------------+
|    | filename                                                                 |   measured_IoU | criterion   |   thresh |   fp |   tp |   fn |   precision |   recall |   accuracy |       f1 |   n_true |   n_pred |   mean_true_score |   mean_matched_score |   panoptic_quality |
|----+--------------------------------------------------------------------------+----------------+